In [ ]:
pricing_hostname = 'https://pricing.api.santecapital.dev'

In [ ]:
from sc_pricing import ScPricing

pricing_client = ScPricing(hostname='https://pricing.api.santecapital.com')

In [ ]:
import sc_trading_cal
import datetime as dt

In [ ]:
from sc_pricing import ScPricing
import os
import datetime as dt
import pandas as pd


class PricingProvider:

    def __init__(self, pricing_client, base_url):
        self.pricing_client = pricing_client
        self.base_url = base_url

    def init_client(self):
        # Access the environment variable for the api key
        # Initialize client
        self.client = self.pricing_client(hostname=self.base_url,
                                          as_of=dt.datetime.now(
                                              dt.timezone.utc),
                                          with_cache=False)

    async def get_price_history(self, trading_item_ids, date_range):
        """
        Format trading_item_ids and return pricing history for a given set of dates
        """
        self.init_client()
        keys = pd.Series(trading_item_ids).apply(lambda x: int(x)).to_list()
        prices = await self.client.get_prices(
            trade_date_range=date_range,
            trading_item_ids=keys,
        )
        await self.client.http_client.close()
        return prices.reset_index()

In [ ]:
pricing_provider = PricingProvider(
        pricing_client=ScPricing,
        base_url='https://pricing.api.santecapital.com',
    )

In [ ]:
from typing import List

In [ ]:
async def get_daily_data(trading_item_ids: List, date_range):
    prices = await pricing_provider.get_price_history(
            trading_item_ids = trading_item_ids, 
            date_range = date_range)
    return prices

In [ ]:
df = await get_daily_data([6179710], sc_trading_cal.get_trade_day_range(start=dt.date(2011, 5, 2), end=dt.date(2022, 2, 18)) )



In [ ]:
df.to_csv('SPX-Price-Daily.csv')

In [ ]:
df

In [ ]:
async def algoseek(year, month, day, tickers):
    '''
    year: int 
    month: int
    tickers : list [str], tickers
    
    '''
    import datetime as dt
    import sc_trading_cal
    from sc_pricing import ScPricing
    pricing_service_host = 'https://pricing.api.santecapital.com'
    pricing_client = ScPricing(hostname= pricing_service_host)

    prices_df = await pricing_client.get_prices(
        source="algoseek",
        trade_date_range=sc_trading_cal.get_trade_day_range(start=dt.date(2021, 11, 13), end=dt.date(2022, 2, 18)),
        resolution='minute',
        adjusted=True,
        adjustment_types=["DIV", "SD"],
        entity_id_type = 'ticker',
        entity_ids= tickers,
    )
    return prices_df

In [ ]:
df2 = await algoseek(2021,11,12, ['SPY'])

In [ ]:
df.to_csv("SPX_minute_2011_05_02-2021-11-12.csv")

In [ ]:
df2